# 01) IMPORT RAW DATA

In [1]:
import math
import pandas as pd
import scipy.stats as stats
import statsmodels.formula.api as sm

In [2]:
# lookup table: NAWQA Wall-to-Wall Anthropogenic Landuse Trends (NWALT) fields (from http://bit.ly/F7XW4J1JDataset8)
strNWALT = 'F7XW4J1J (Anthropogenic Influences)/NWALT.csv'

In [3]:
# import lookup table
dfNWALT = pd.read_csv(strNWALT)

In [4]:
# define paths to source files (from http://bit.ly/F7XW4J1JDataset8)
strLand = 'F7XW4J1J (Anthropogenic Influences)/Land Use Change, 1982-2012.csv'
strPH = 'Change in Mean pH Levels 82-12.csv'

In [5]:
# import data
dfLand = pd.read_csv(strLand)
dfPH = pd.read_csv(strPH)

In [6]:
# describe Land dataframe
dfLand.describe()

,GEOID5,Chg-Water,Chg-Wetlands,Chg-Dev-Trans,Chg-Dev-CommSvcs,Chg-Dev-IndMil,Chg-Dev-Recr,Chg-Dev-ResHi,Chg-Dev-ResLoMed,Chg-Dev-Other,...,Chg-SemiDev-UrbIntLoMed,Chg-SemiDev-Other,Chg-Mining,Chg-Crops,Chg-Pasture,Chg-Grazing1,Chg-Grazing2,Chg-LowUse,Chg-VeryLowUse,Chg-Unknown
count,3109.000000,3109.000000,3109.0,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,...,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.0
mean,30678.541975,0.038112,0.0,0.114130,0.193654,0.092461,0.091560,0.331492,0.645954,-0.180167,...,2.178604,-0.004397,0.029781,-0.311618,-0.806739,0.043628,0.000495,-2.627533,0.336131,0.0
std,14986.022374,0.205478,0.0,0.227155,0.602751,0.213809,0.216703,1.260709,1.635504,0.403690,...,4.839980,0.015414,0.140768,1.795365,1.852995,0.305452,0.626873,5.785244,2.096985,0.0
min,1001.000000,-0.100000,0.0,0.000000,0.000000,-1.360000,0.000000,-0.200000,-8.650000,-4.180000,...,-24.290000,-0.250000,-1.270000,-14.710000,-12.280000,-3.570000,-11.410000,-59.600000,-0.010000,0.0
25%,19045.000000,0.000000,0.0,0.030000,0.010000,0.000000,0.000000,0.000000,0.010000,-0.200000,...,0.000000,0.000000,0.000000,-0.890000,-1.500000,-0.010000,-0.030000,-3.320000,0.000000,0.0
50%,29213.000000,0.000000,0.0,0.060000,0.030000,0.020000,0.020000,0.010000,0.090000,0.000000,...,0.110000,0.000000,0.000000,-0.080000,-0.410000,0.010000,0.000000,-0.870000,0.000000,0.0
75%,46009.000000,0.010000,0.0,0.130000,0.120000,0.080000,0.080000,0.100000,0.570000,0.020000,...,2.010000,0.000000,0.020000,0.460000,0.000000,0.090000,0.090000,0.000000,0.000000,0.0
max,56045.000000,5.500000,0.0,7.120000,16.120000,2.510000,3.520000,31.960000,25.780000,0.860000,...,57.840000,0.120000,2.570000,7.320000,10.070000,2.850000,3.080000,9.160000,56.350000,0.0


In [7]:
# describe pH dataframe
dfPH.describe()

,FIPS Code,1982 Mean pH Value,2012 Mean pH Value
count,1450.000000,1450.000000,1450.000000
mean,29619.763448,7.471949,7.664822
std,17328.743233,0.792767,0.838485
min,1001.000000,2.200000,3.927763
25%,17008.000000,7.154911,7.350229
50%,29106.000000,7.600000,7.755990
75%,42033.500000,7.914815,8.054307
max,72151.000000,21.162361,30.345930


# 02) REFORMAT DATA

In [8]:
# rename dfLand's "GEOID5" col for merging
dictCol = {'GEOID5':'FIPS Code'}
for col in dfLand.filter(regex='Chg').columns:
    dictCol[col]=col.replace('-','') 
dfLand = dfLand.rename(columns=dictCol)
dfLand.describe()

,FIPS Code,ChgWater,ChgWetlands,ChgDevTrans,ChgDevCommSvcs,ChgDevIndMil,ChgDevRecr,ChgDevResHi,ChgDevResLoMed,ChgDevOther,...,ChgSemiDevUrbIntLoMed,ChgSemiDevOther,ChgMining,ChgCrops,ChgPasture,ChgGrazing1,ChgGrazing2,ChgLowUse,ChgVeryLowUse,ChgUnknown
count,3109.000000,3109.000000,3109.0,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,...,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.0
mean,30678.541975,0.038112,0.0,0.114130,0.193654,0.092461,0.091560,0.331492,0.645954,-0.180167,...,2.178604,-0.004397,0.029781,-0.311618,-0.806739,0.043628,0.000495,-2.627533,0.336131,0.0
std,14986.022374,0.205478,0.0,0.227155,0.602751,0.213809,0.216703,1.260709,1.635504,0.403690,...,4.839980,0.015414,0.140768,1.795365,1.852995,0.305452,0.626873,5.785244,2.096985,0.0
min,1001.000000,-0.100000,0.0,0.000000,0.000000,-1.360000,0.000000,-0.200000,-8.650000,-4.180000,...,-24.290000,-0.250000,-1.270000,-14.710000,-12.280000,-3.570000,-11.410000,-59.600000,-0.010000,0.0
25%,19045.000000,0.000000,0.0,0.030000,0.010000,0.000000,0.000000,0.000000,0.010000,-0.200000,...,0.000000,0.000000,0.000000,-0.890000,-1.500000,-0.010000,-0.030000,-3.320000,0.000000,0.0
50%,29213.000000,0.000000,0.0,0.060000,0.030000,0.020000,0.020000,0.010000,0.090000,0.000000,...,0.110000,0.000000,0.000000,-0.080000,-0.410000,0.010000,0.000000,-0.870000,0.000000,0.0
75%,46009.000000,0.010000,0.0,0.130000,0.120000,0.080000,0.080000,0.100000,0.570000,0.020000,...,2.010000,0.000000,0.020000,0.460000,0.000000,0.090000,0.090000,0.000000,0.000000,0.0
max,56045.000000,5.500000,0.0,7.120000,16.120000,2.510000,3.520000,31.960000,25.780000,0.860000,...,57.840000,0.120000,2.570000,7.320000,10.070000,2.850000,3.080000,9.160000,56.350000,0.0


In [9]:
# create numeric "pH Improvement" column
dfPH['ChgPHQual'] = abs(7.25 - dfPH['1982 Mean pH Value']) - abs(7.25 - dfPH['2012 Mean pH Value'])
dfPH.describe()

,FIPS Code,1982 Mean pH Value,2012 Mean pH Value,ChgPHQual
count,1450.000000,1450.000000,1450.000000,1450.000000
mean,29619.763448,7.471949,7.664822,-0.037192
std,17328.743233,0.792767,0.838485,0.902607
min,1001.000000,2.200000,3.927763,-22.908430
25%,17008.000000,7.154911,7.350229,-0.354405
50%,29106.000000,7.600000,7.755990,-0.055739
75%,42033.500000,7.914815,8.054307,0.229425
max,72151.000000,21.162361,30.345930,13.731488


In [10]:
# describe new dataframe
dfMrg = pd.merge(dfLand, dfPH, on='FIPS Code')
dfMrg.describe()

,FIPS Code,ChgWater,ChgWetlands,ChgDevTrans,ChgDevCommSvcs,ChgDevIndMil,ChgDevRecr,ChgDevResHi,ChgDevResLoMed,ChgDevOther,...,ChgCrops,ChgPasture,ChgGrazing1,ChgGrazing2,ChgLowUse,ChgVeryLowUse,ChgUnknown,1982 Mean pH Value,2012 Mean pH Value,ChgPHQual
count,1390.000000,1390.000000,1390.0,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,...,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.0,1390.000000,1390.000000,1390.000000
mean,28470.271223,0.042727,0.0,0.119165,0.217683,0.104245,0.111698,0.400360,0.806885,-0.194698,...,-0.413043,-0.847014,0.051489,-0.010007,-2.995583,0.507791,0.0,7.464941,7.647349,-0.018752
std,15694.842378,0.222042,0.0,0.234975,0.515088,0.206504,0.236787,1.153719,1.795725,0.411671,...,1.791740,1.744780,0.271467,0.627258,6.032287,2.766590,0.0,0.804258,0.596387,0.683955
min,1001.000000,-0.070000,0.0,0.000000,0.000000,0.000000,0.000000,-0.200000,-4.110000,-4.180000,...,-10.520000,-12.280000,-3.570000,-9.870000,-56.420000,-0.010000,0.0,2.200000,3.927763,-1.697755
25%,17004.000000,0.000000,0.0,0.030000,0.010000,0.000000,0.000000,0.000000,0.010000,-0.230000,...,-0.990000,-1.480000,-0.010000,-0.040000,-3.807500,0.000000,0.0,7.139174,7.338573,-0.360890
50%,29022.000000,0.000000,0.0,0.070000,0.040000,0.020000,0.020000,0.010000,0.110000,-0.010000,...,-0.120000,-0.405000,0.010000,0.000000,-1.080000,0.000000,0.0,7.600000,7.756498,-0.054157
75%,40116.500000,0.020000,0.0,0.140000,0.170000,0.107500,0.100000,0.160000,0.767500,0.010000,...,0.320000,0.010000,0.090000,0.080000,-0.092500,0.000000,0.0,7.915506,8.061421,0.237573
max,56041.000000,5.500000,0.0,7.120000,6.940000,1.880000,2.610000,10.210000,25.780000,0.530000,...,7.320000,6.990000,2.740000,2.670000,7.660000,56.350000,0.0,21.162361,9.029167,13.731488


# 03) COMPUTE LAND USE-TO-pH CORRELATIONS

In [11]:
# create format strings to prettify output
strFmt = '{:,.3f}'
strFmtSignSpc = '{:+1,.3f}'
strFmtSpc = '{: ,.3f}'
# compute regressions against each var
for col in dfMrg.filter(regex='Chg').columns:
    slope, intercept, r_value, p_value, std_err=stats.linregress(dfMrg[col], dfMrg['ChgPHQual'])
    strMX = f'{strFmtSpc.format(slope)} × {col.ljust(21)}'
    strB = f'{strFmtSignSpc.format(intercept)}'
    strPAndR2 = f'P = {strFmt.format(p_value)}, R² = {strFmtSignSpc.format(r_value**2)}'
    if not math.isnan(slope):
        print(f'ChgPHQual = {strB} + {strMX} | {strPAndR2}')

ChgPHQual = -0.020 +  0.033 × ChgWater              | P = 0.689, R² = +0.000
ChgPHQual = -0.032 +  0.114 × ChgDevTrans           | P = 0.143, R² = +0.002
ChgPHQual = -0.037 +  0.085 × ChgDevCommSvcs        | P = 0.018, R² = +0.004
ChgPHQual = -0.039 +  0.191 × ChgDevIndMil          | P = 0.031, R² = +0.003
ChgPHQual = -0.042 +  0.205 × ChgDevRecr            | P = 0.008, R² = +0.005
ChgPHQual = -0.047 +  0.071 × ChgDevResHi           | P = 0.000, R² = +0.014
ChgPHQual = -0.033 +  0.018 × ChgDevResLoMed        | P = 0.078, R² = +0.002
ChgPHQual = -0.049 + -0.158 × ChgDevOther           | P = 0.000, R² = +0.009
ChgPHQual = -0.034 + -0.069 × ChgSemiDevUrbIntHi    | P = 0.005, R² = +0.006
ChgPHQual = -0.045 +  0.012 × ChgSemiDevUrbIntLoMed | P = 0.002, R² = +0.007
ChgPHQual = -0.026 + -1.488 × ChgSemiDevOther       | P = 0.220, R² = +0.001
ChgPHQual = -0.022 +  0.103 × ChgMining             | P = 0.514, R² = +0.000
ChgPHQual = -0.031 + -0.029 × ChgCrops              | P = 0.004, R² = +0.006

C:\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:116: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
C:\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:128: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [12]:
result = sm.ols(formula='ChgPHQual ~ ChgWater + ChgDevTrans + ChgDevCommSvcs + ChgDevIndMil + ChgDevRecr + ChgDevResHi + ChgDevResLoMed + ChgDevOther + ChgSemiDevUrbIntHi + ChgSemiDevUrbIntLoMed + ChgSemiDevOther + ChgMining + ChgCrops + ChgPasture + ChgGrazing1 + ChgGrazing2 + ChgLowUse + ChgVeryLowUse', data=dfMrg).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              ChgPHQual   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     3.705
Date:                Thu, 04 Jul 2019   Prob (F-statistic):           2.57e-07
Time:                        08:33:04   Log-Likelihood:                -1410.8
No. Observations:                1390   AIC:                             2860.
Df Residuals:                    1371   BIC:                             2959.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                -0.05

# 04) MAP pH BY COUNTY

In [13]:
# import plotly and set credentials
from PlotlyConfig import un, pkey 
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.io as pio
import numpy as np
plotly.tools.set_credentials_file(username=un, api_key=pkey)

In [14]:
def MakeCtyFig(strVal, strTitle='Water Quality', strLegend='', fRound=False, fBin=True, strFIPS='FIPS Code', df=dfPH):
    # create lists of parameter values
    lstFIPS = df[strFIPS].tolist()
    lstVal = df[strVal].tolist()
    if fRound:
        lstVal = [round(val, 2) for val in lstVal]
    lstColor = ['#FF0040','#FF0000','#FF2800','#FF5000','#FF7800', \
                '#FFa000','#FFc800','#FFf000','#b0ff00','#17ff00', \
                '#00ff83','#00e4ff','#00a4ff','#0064ff','#0022ff', \
                '#0100ff','#0500ff'] # 13-color ROYGB
    intColor = len(lstColor)
    intBinSize = 1 / (intColor + 1) * 100
    lstBin = list(np.linspace(np.percentile(lstVal, intBinSize), \
                              np.percentile(lstVal, 100 - intBinSize), \
                              intColor - 1))
    # set fig variable
    if fBin:
        fig = ff.create_choropleth(
            fips=lstFIPS, values=lstVal,
            binning_endpoints=lstBin,
            colorscale=lstColor,
            show_state_data=False,
            show_hover=True, centroid_marker={'opacity': 0},
            asp=2.9, title=strTitle,
            legend_title=strLegend
        )
    else:
        fig = ff.create_choropleth(
            fips=lstFIPS, values=lstVal,
            title=strTitle,
            legend_title=strLegend
        )
        
    return fig

In [15]:
fig = MakeCtyFig('ChgPHQual', 'Progress toward Ideal pH (7.25), 1982-2012', 'Net pH Improvement', True)
py.iplot(fig, filename='choropleth_full_usa')

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


C:\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [16]:
pio.write_image(fig, 'Station_List/pH by County 1982-2012.png')

# 05) CREATE WATER QUALITY FUNCTION

### Water Quality Index computed using http://www.pathfinderscience.net/stream/forms/WQI_worksheet.pdf

In [17]:
# create Dictionaries to compute "Q-Values" according to http://www.pathfinderscience.net/stream/cproto4.cfm
# convert Fecal Coliform to Q-value
#     http://www.pathfinderscience.net/stream/forms/WQI_FC.pdf
dictFC2Q = {1:97, 2:90, 5:81, 10:72, 20:62, 50:55, 100:46, 200:37, 500:27, \
            1000:22, 2000:18, 5000:15, 10000:9, 20000:7, 50000:5, 100000:4}
# convert pH to Q-value   
#     http://www.pathfinderscience.net/stream/forms/WQI_pH.pdf
dictPH2Q = {2:0, 3:4, 4:9, 5:28, 6:55, 7:89, 7.5:93, 8:85, 9:49, 10:20, 11:7, 12:2}
# convert Turbidity to Q-value
#     http://www.pathfinderscience.net/stream/forms/WQI_Turb.pdf
dictTurb2Q = {0:98, 10:77, 20:61, 30:53, 40:46, 50:38, 60:34, 70:29, 80:25, 90:21, 100:17, 100.01:5, 1000000:5}

In [18]:
# define function that computes approximate Q-values
def EstimateQ(fltArg, dictLkp):
    # create dataframe for Arg, the argument we want to convert to a Q-value
    dfArg = pd.DataFrame(columns=['Arg'])
    dfArg['Arg'] = dfArg['Arg'].astype(float)
    dfArg.loc[0, 'Arg'] = fltArg
    # create dataframe for the dictionary we're going to look Arg up in
    dfLkp = pd.DataFrame.from_dict(dictLkp, orient='index', columns=['Q'])
    dfLkp = dfLkp.reset_index()
    dfLkp = dfLkp.rename(columns={'index':'Arg'})
    dfLkp['Arg']=dfLkp['Arg'].astype(float)
    dfLkp['OrigArg'] = dfLkp['Arg']
    # use merge_asof() with 'backward' param to find the low lookup value
    dfLkpLo = pd.merge_asof(dfArg, dfLkp, on='Arg', direction='backward')
    fltArgLo = dfLkpLo.loc[0,'OrigArg']
    fltQLo = dfLkpLo.loc[0,'Q']
    # use merge_asof() with 'forward' param to find the high lookup value
    dfLkpHi = pd.merge_asof(dfArg, dfLkp, on='Arg', direction='forward')
    fltArgHi = dfLkpHi.loc[0,'OrigArg']
    fltQHi = dfLkpHi.loc[0,'Q']
    # determine Q-value
    if fltArg == fltArgHi:
        fltQ = fltQHi
    elif fltArg == fltArgLo:
        fltQ = fltQLo
    else: # compute weighted Q-value between high and low lookups
        fltQ = ((fltArgHi - fltArg) * fltQLo + (fltArg - fltArgLo) * fltQHi) / (fltArgHi - fltArgLo)
    # return a list showing 
    # 1) Q, and the 2 Q-values it's weighted between
    # 2) Arg, and the 2 Args it's weighted between
    return [[fltQ, 'QLo='+str(fltQLo), 'QHi='+str(fltQHi)], \
            [fltArg, 'ArgLo='+str(fltArgLo), 'ArgHi='+str(fltArgHi)]]

In [19]:
# test the EstimateQ function. In the Fecal Coliform dictionary, an FC of 300 should 
# apply .67 weight to FC=200's Q-value of 37 and apply .33 weight to FC=500's Q-value of 27.
EstimateQ(300, dictFC2Q)

[[33.666666666666664, 'QLo=37', 'QHi=27'], [300, 'ArgLo=200.0', 'ArgHi=500.0']]

In [20]:
# test EstimateQ, but just return the Q-value
EstimateQ(300, dictFC2Q)[0][0]

33.666666666666664

In [21]:
# establish weights for each Q-value: http://www.pathfinderscience.net/stream/forms/WQI_worksheet.pdf
fltWtFC = float(16)  # weight: fecal coliform
fltWtPH = float(11)  # weight: pH
fltWtTurb = float(8) # weight: turbidity

In [22]:
# define function that computes weighted-avg Water Quality
def ComputeWQI(fltFC=math.nan, fltPH=math.nan, fltTurb=math.nan):
    # establish Q-value vars
    fltQFC, fltQPH, fltQTurb = math.nan, math.nan, math.nan
    # establish numerator & denominator vars
    fltNum, fltDen = 0, 0
    if not math.isnan(fltFC):
        fltQFC = EstimateQ(fltFC, dictFC2Q)[0][0]
        fltNum += fltWtFC * fltQFC
        fltDen += fltWtFC
    if not math.isnan(fltPH):
        fltQPH = EstimateQ(fltPH, dictPH2Q)[0][0]
        fltNum += fltWtPH * fltQPH
        fltDen += fltWtPH
    if not math.isnan(fltTurb):
        fltQTurb = EstimateQ(fltTurb, dictTurb2Q)[0][0]
        fltNum += fltWtTurb * fltQTurb
        fltDen += fltWtTurb
    return [fltNum // fltDen, \
            ['FC='+str(fltQFC), 'pH='+str(fltQPH), 'Turb='+str(fltQTurb)], \
            ['WtFC='+str(fltWtFC), 'WtPH='+str(fltWtPH), 'WtTurb='+str(fltWtTurb)]]

In [23]:
# test the ComputeWQ function: wtg avg for two inputs, with missing arg
ComputeWQI(7, 9.25)

[62.0,
 ['FC=77.4', 'pH=41.75', 'Turb=nan'],
 ['WtFC=16.0', 'WtPH=11.0', 'WtTurb=8.0']]

In [24]:
# test ComputeWQ function, but just return the WQ index
ComputeWQI(7, 9.25)[0]

62.0

# 06) MAP 2012 WATER QUALITY BY COUNTY

In [25]:
# create path to WQ2012 file
strWQ2012 = '2012_Water_Quality_df.csv'

In [26]:
# import WQ2012 file
dfWQ2012 = pd.read_csv(strWQ2012)

In [27]:
# peek at WQ2012
dfWQ2012.head()

,StateCode,CountyCode,FIPS Code,Mean Temperature (c),Mean_pH,Mean Turbidity,Mean Fecal Coliform,Mean Flow
0,1,1,01001,18.7,6.1,6.3,NaN,27.9
1,1,3,01003,23.9,6.6,9.4,49.7,199.2
2,1,5,01005,25.9,8.2,5.5,NaN,5.7
3,1,7,01007,20.4,7.1,45.4,NaN,319.4
4,1,9,01009,22.2,7.7,4.6,NaN,48.8


In [28]:
# describe WQ2012
dfWQ2012.describe()

,StateCode,Mean Temperature (c),Mean_pH,Mean Turbidity,Mean Fecal Coliform,Mean Flow
count,2473.000000,2472.000000,2405.000000,1954.000000,875.000000,647.000000
mean,30.999596,18.400594,7.612724,40.211975,1331.586008,1068.262092
std,16.115745,10.311329,1.952590,524.886579,9732.030159,3944.226718
min,1.000000,1.000000,-42.200000,-1.186389,1.000000,0.000000
25%,19.000000,14.100000,7.300000,4.100000,61.550000,8.056299
50%,30.000000,17.500000,7.700000,9.700000,184.700000,48.800000
75%,45.000000,21.200000,8.077500,22.600000,411.600000,333.900000
max,78.000000,393.400000,42.500000,21849.300000,206073.200000,32492.800000


In [29]:
# drop columns not used in computation:
# --Temperature requires two readings to determine Difference in Temps
# --Flow is not used in the methodology in http://www.pathfinderscience.net/stream/cproto4.cfm
dfWQ2012 = dfWQ2012.drop('Mean Temperature (c)', axis=1)
dfWQ2012 = dfWQ2012.drop('Mean Flow', axis=1)

In [30]:
# drop bad rows (where County is undefined)
dfWQ2012 = dfWQ2012.loc[dfWQ2012['CountyCode'] != '(blank)']

In [31]:
# drop bad rows (where there are no valid values)
dfWQ2012 = dfWQ2012.dropna(subset=['Mean Fecal Coliform', 'Mean_pH', 'Mean Turbidity'], thresh=1)

In [32]:
# create "Fill" columns, replacing blanks with median values
dfWQ2012['Mean Fecal Coliform Fill'] = dfWQ2012['Mean Fecal Coliform'].fillna(dfWQ2012['Mean Fecal Coliform'].median())
dfWQ2012['Mean_pH Fill'] = dfWQ2012['Mean_pH'].fillna(dfWQ2012['Mean_pH'].median())
dfWQ2012['Mean Turbidity Fill'] = dfWQ2012['Mean Turbidity'].fillna(dfWQ2012['Mean Turbidity'].median())

In [33]:
dfWQ2012.describe()

,StateCode,Mean_pH,Mean Turbidity,Mean Fecal Coliform,Mean Fecal Coliform Fill,Mean_pH Fill,Mean Turbidity Fill
count,2413.000000,2402.000000,1953.000000,874.000000,2413.000000,2413.000000,2413.000000
mean,31.124327,7.612576,40.230927,1333.098289,600.719065,7.612974,34.410692
std,16.150216,1.953750,525.020341,9737.499593,5884.175145,1.949298,472.462558
min,1.000000,-42.200000,-1.186389,1.000000,1.000000,-42.200000,-1.186389
25%,19.000000,7.300000,4.100000,62.075000,184.800000,7.300000,5.100000
50%,30.000000,7.700000,9.700000,184.800000,184.800000,7.700000,9.700000
75%,45.000000,8.077492,22.600000,411.750000,184.800000,8.075000,18.000000
max,78.000000,42.500000,21849.300000,206073.200000,206073.200000,42.500000,21849.300000


In [34]:
# change FIPS Code to float
dfWQ2012['FIPS Code'] = dfWQ2012['FIPS Code'].astype('float')

In [35]:
# compute WQ for all rows
for index, row in dfWQ2012.iterrows():
    dfWQ2012.loc[index, 'WQIFill'] = \
     ComputeWQI(row['Mean Fecal Coliform Fill'], row['Mean_pH Fill'], row['Mean Turbidity Fill'])[0]
    dfWQ2012.loc[index, 'WQI'] = \
     ComputeWQI(row['Mean Fecal Coliform'], row['Mean_pH'], row['Mean Turbidity'])[0]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in double_scalars



In [36]:
# merge WQ2012 into dfMrg and describe
dfMrg = pd.merge(dfMrg, dfWQ2012, on='FIPS Code')
dfMrg.describe()

,FIPS Code,ChgWater,ChgWetlands,ChgDevTrans,ChgDevCommSvcs,ChgDevIndMil,ChgDevRecr,ChgDevResHi,ChgDevResLoMed,ChgDevOther,...,ChgPHQual,StateCode,Mean_pH,Mean Turbidity,Mean Fecal Coliform,Mean Fecal Coliform Fill,Mean_pH Fill,Mean Turbidity Fill,WQIFill,WQI
count,1382.000000,1382.000000,1382.0,1382.000000,1382.000000,1382.000000,1382.000000,1382.000000,1382.000000,1382.000000,...,1382.000000,1382.000000,1381.000000,1139.000000,473.000000,1382.000000,1382.000000,1382.000000,1376.000000,1376.000000
mean,28520.913893,0.042974,0.0,0.119385,0.218502,0.104559,0.111961,0.401368,0.810463,-0.195745,...,-0.018106,28.438495,7.593572,55.385495,998.135108,463.170120,7.593649,47.352517,60.011628,71.799419
std,15699.735606,0.222661,0.0,0.235579,0.516406,0.206982,0.237349,1.156563,1.800163,0.412590,...,0.685658,15.697361,2.080515,686.663569,8243.502513,4834.758485,2.079764,623.573132,8.076625,13.527256
min,1001.000000,-0.070000,0.0,0.000000,0.000000,0.000000,0.000000,-0.200000,-4.110000,-4.180000,...,-1.697755,1.000000,-42.200000,0.100000,1.000000,1.000000,-42.200000,0.100000,33.000000,33.000000
25%,17008.000000,0.000000,0.0,0.030000,0.010000,0.000000,0.000000,0.000000,0.010000,-0.230000,...,-0.360890,17.000000,7.300000,4.400000,51.300000,184.800000,7.300000,5.300000,55.000000,61.000000
50%,29042.000000,0.000000,0.0,0.070000,0.040000,0.020000,0.020000,0.010000,0.110000,-0.010000,...,-0.052123,29.000000,7.800000,10.400000,179.800000,184.800000,7.800000,9.700000,60.000000,73.000000
75%,40118.500000,0.020000,0.0,0.140000,0.170000,0.107500,0.100000,0.160000,0.777500,0.010000,...,0.238986,40.000000,8.100000,25.000000,412.400000,184.800000,8.100000,19.400000,64.000000,83.000000
max,56041.000000,5.500000,0.0,7.120000,6.940000,1.880000,2.610000,10.210000,25.780000,0.530000,...,13.731488,56.000000,26.000000,21849.300000,170000.000000,170000.000000,26.000000,21849.300000,90.000000,94.000000


In [37]:
dfMrg = dfMrg.dropna(subset=['WQIFill'])

In [38]:
fig = MakeCtyFig('WQIFill', strTitle='Water Quality, 2012 (with Fill)', strLegend='Water Quality Index', df=dfMrg)
py.iplot(fig, filename='choropleth_full_usa')

C:\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [39]:
dfMrg = dfMrg.dropna(subset=['WQI'])

In [40]:
fig = MakeCtyFig('WQI', strTitle='Water Quality, 2012 (without Fill)', strLegend='Water Quality Index', df=dfMrg)
py.iplot(fig, filename='choropleth_full_usa')

C:\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



# 07) COMPUTE LAND USE-TO-WATER QUALITY CORRELATIONS

In [41]:
# compute regressions against each var: with Fill
for col in dfMrg.filter(regex='Chg').columns:
    slope, intercept, r_value, p_value, std_err=stats.linregress(dfMrg[col], dfMrg['WQIFill'])
    strMX = f'{strFmtSpc.format(slope).rjust(7)} × {col.ljust(21)}'
    strB = f'{strFmtSignSpc.format(intercept)}'
    strPAndR2 = f'P = {strFmt.format(p_value)}, R² = {strFmtSignSpc.format(r_value**2)}'
    if not math.isnan(slope):
        print(f'Water Quality 2012 (with Fill) = {strB} + {strMX} | {strPAndR2}')

Water Quality 2012 (with Fill) = +60.046 +  -0.806 × ChgWater              | P = 0.409, R² = +0.000
Water Quality 2012 (with Fill) = +59.908 +   0.865 × ChgDevTrans           | P = 0.349, R² = +0.001
Water Quality 2012 (with Fill) = +60.004 +   0.033 × ChgDevCommSvcs        | P = 0.938, R² = +0.000
Water Quality 2012 (with Fill) = +59.927 +   0.809 × ChgDevIndMil          | P = 0.442, R² = +0.000
Water Quality 2012 (with Fill) = +59.809 +   1.800 × ChgDevRecr            | P = 0.049, R² = +0.003
Water Quality 2012 (with Fill) = +59.995 +   0.041 × ChgDevResHi           | P = 0.826, R² = +0.000
Water Quality 2012 (with Fill) = +59.832 +   0.221 × ChgDevResLoMed        | P = 0.067, R² = +0.002
Water Quality 2012 (with Fill) = +59.460 +  -2.822 × ChgDevOther           | P = 0.000, R² = +0.021
Water Quality 2012 (with Fill) = +60.021 +   0.044 × ChgSemiDevUrbIntHi    | P = 0.879, R² = +0.000
Water Quality 2012 (with Fill) = +59.579 +   0.189 × ChgSemiDevUrbIntLoMed | P = 0.000, R² = +0.013


C:\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:116: RuntimeWarning:

invalid value encountered in double_scalars

C:\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:128: RuntimeWarning:

divide by zero encountered in double_scalars



In [42]:
result = sm.ols(formula='WQIFill ~ ChgWater + ChgDevTrans + ChgDevCommSvcs + ChgDevIndMil + ChgDevRecr + ChgDevResHi + ChgDevResLoMed + ChgDevOther + ChgSemiDevUrbIntHi + ChgSemiDevUrbIntLoMed + ChgSemiDevOther + ChgMining + ChgCrops + ChgPasture + ChgGrazing1 + ChgGrazing2 + ChgLowUse + ChgVeryLowUse', data=dfMrg).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                WQIFill   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     5.996
Date:                Thu, 04 Jul 2019   Prob (F-statistic):           3.35e-14
Time:                        08:39:11   Log-Likelihood:                -4773.7
No. Observations:                1376   AIC:                             9585.
Df Residuals:                    1357   BIC:                             9685.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                59.85

In [43]:
# compute regressions against each var: without Fill
for col in dfMrg.filter(regex='Chg').columns:
    slope, intercept, r_value, p_value, std_err=stats.linregress(dfMrg[col], dfMrg['WQI'])
    strMX = f'{strFmtSpc.format(slope).rjust(7)} × {col.ljust(21)}'
    strB = f'{strFmtSignSpc.format(intercept)}'
    strPAndR2 = f'P = {strFmt.format(p_value)}, R² = {strFmtSignSpc.format(r_value**2)}'
    if not math.isnan(slope):
        print(f'Water Quality 2012 (without Fill) = {strB} + {strMX} | {strPAndR2}')

Water Quality 2012 (without Fill) = +71.810 +  -0.235 × ChgWater              | P = 0.886, R² = +0.000
Water Quality 2012 (without Fill) = +72.030 +  -1.925 × ChgDevTrans           | P = 0.213, R² = +0.001
Water Quality 2012 (without Fill) = +72.307 +  -2.314 × ChgDevCommSvcs        | P = 0.001, R² = +0.008
Water Quality 2012 (without Fill) = +72.268 +  -4.463 × ChgDevIndMil          | P = 0.011, R² = +0.005
Water Quality 2012 (without Fill) = +72.077 +  -2.471 × ChgDevRecr            | P = 0.107, R² = +0.002
Water Quality 2012 (without Fill) = +71.996 +  -0.488 × ChgDevResHi           | P = 0.121, R² = +0.002
Water Quality 2012 (without Fill) = +72.277 +  -0.588 × ChgDevResLoMed        | P = 0.004, R² = +0.006
Water Quality 2012 (without Fill) = +71.908 +   0.555 × ChgDevOther           | P = 0.531, R² = +0.000
Water Quality 2012 (without Fill) = +72.006 +   0.957 × ChgSemiDevUrbIntHi    | P = 0.050, R² = +0.003
Water Quality 2012 (without Fill) = +71.928 +  -0.056 × ChgSemiDevUrbIntL

In [44]:
result = sm.ols(formula='WQI ~ ChgWater + ChgDevTrans + ChgDevCommSvcs + ChgDevIndMil + ChgDevRecr + ChgDevResHi + ChgDevResLoMed + ChgDevOther + ChgSemiDevUrbIntHi + ChgSemiDevUrbIntLoMed + ChgSemiDevOther + ChgMining + ChgCrops + ChgPasture + ChgGrazing1 + ChgGrazing2 + ChgLowUse + ChgVeryLowUse', data=dfMrg).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    WQI   R-squared:                       0.058
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     4.655
Date:                Thu, 04 Jul 2019   Prob (F-statistic):           4.18e-10
Time:                        08:39:11   Log-Likelihood:                -5494.8
No. Observations:                1376   AIC:                         1.103e+04
Df Residuals:                    1357   BIC:                         1.113e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                72.40